In [1]:
!ls ../3000000to3999999_BlockTransaction/

3000000to3999999_BlockTransaction.csv  final.csv      readme.txt  tx_counts.csv
extracted_labels.csv		       processed.csv  time.csv


In [2]:
path = '../3000000to3999999_BlockTransaction/'

In [3]:
import pandas as pd
from datetime import datetime
import time
import numpy as np

In [4]:
df = pd.read_csv(path+'final.csv')

In [5]:
df.head()

,timestamp,from,to,value,gasLimit,gasPrice,gasUsed,0
0,1484475035,0,1890222,4.950000,3.9,2.0,2.1,mining
1,1484475035,1,1777,1.011962,9.0,2.0,2.1,mining
2,1484475035,1,20425,1.011957,9.0,2.0,2.1,mining
3,1484475035,1,1558,1.008363,9.0,2.0,2.1,mining
4,1484475035,1,15,1.016120,9.0,2.0,2.1,mining


In [6]:
df['0'].unique()

array(['mining', nan, 'exchange', 'walletapp', 'compromised', 'gambling',
       'isowallet', 'phish_hack'], dtype=object)

In [7]:
labels = df['0'].unique()
for label in labels:
    print('%s: '% label,(df['0'] == label).sum())
# print('mining: ',(merged_df['0'] == 'mining').sum())

mining:  3499995
nan:  0
exchange:  1315278
walletapp:  52765
compromised:  24937
gambling:  823
isowallet:  135
phish_hack:  172


In [8]:
labels[2]

'exchange'

In [9]:
print(labels[2])
exchange_data = df[df['0'] == labels[2]]
exchange_data.head()

exchange


,timestamp,from,to,value,gasLimit,gasPrice,gasUsed,0
22,1484475182,13,1676,0.989400,33.3333,3.0,2.1,exchange
26,1484475182,13,3522,2500.000000,33.3333,3.0,2.1,exchange
37,1484475230,17,54938,25.237936,4.0000,6.0,2.1,exchange
80,1484475418,33,1890243,0.010000,9.0000,2.0,2.1,exchange
84,1484475449,17,55130,0.322861,4.0000,6.0,2.1,exchange


In [10]:
exchange_addrs = exchange_data['from'].unique()

In [11]:
exchange_addrs

array([     13,      17,      33,      34,      43,      54,      56,
           103,     117,     223,     995,    1228,    1656,    1978,
          2991,    5776,   12263,   14671,   18658,   33689,   47074,
         49448,  221195,  429754,  472163,  783594,  799193,  815148,
        831223,  831265,  831328,  831609,  862493,  868818,  876589,
        886250,  899622,  914200,  943390, 1204532, 1214774, 1390781,
       1442627, 1445427, 1445460, 1467198, 1637691, 1643544, 1722452,
       1794274])

In [12]:
kept_ex_addrs = []
for addr in exchange_addrs:
    if (exchange_data['from']==addr).sum() > 100:
        kept_ex_addrs.append(addr)
#     print( str(addr)+' :',(exchange_data['from']==addr).sum())
    

In [13]:
len(kept_ex_addrs)

33

In [14]:
len(exchange_addrs)

50

In [15]:
print(labels[0])
mining_data = df[df['0'] == labels[0]]
mining_data.head()

mining


,timestamp,from,to,value,gasLimit,gasPrice,gasUsed,0
0,1484475035,0,1890222,4.950000,3.9,2.0,2.1,mining
1,1484475035,1,1777,1.011962,9.0,2.0,2.1,mining
2,1484475035,1,20425,1.011957,9.0,2.0,2.1,mining
3,1484475035,1,1558,1.008363,9.0,2.0,2.1,mining
4,1484475035,1,15,1.016120,9.0,2.0,2.1,mining


In [18]:
mining_addrs = mining_data['from'].unique()

In [19]:
len(mining_addrs)

12

In [20]:
kept_mining_addrs = []
for addr in mining_addrs:
    if (mining_data['from']==addr).sum() > 100:
        kept_mining_addrs.append(addr)
#     print( str(addr)+' :',(mining_data['from']==addr).sum())
    

In [21]:
len(kept_mining_addrs)

9

In [22]:
kept_mining_addrs

[0, 1, 261, 338, 1236, 1484, 3402, 76929, 1459688]

In [23]:
def receiving_transaction_graph_h4_timed(node,st_time,end_time,dataframe):
    
    timed_df = dataframe[dataframe['timestamp'].between(st_time,end_time)]
    
    neigh1 = timed_df[timed_df['to'] == node]
    
    neigh2 = timed_df[timed_df['to'].isin(neigh1['from'])]
    neigh2=neigh2[neigh2['to'] != node]
    
    neigh3 = timed_df[timed_df['to'].isin(neigh2['from'])]
    neigh3 = neigh3[~neigh3['to'].isin(neigh2['to'])]
    
    neigh4 = timed_df[timed_df['to'].isin(neigh3['from'])]
    neigh4 = neigh4[~neigh4['to'].isin(neigh3['to'])]
    
    return (neigh1,neigh2,neigh3,neigh4)
    
    

In [24]:
def send_extract_graph_h4_timed(node,st_time,end_time,dataframe):
    
    timed_df = dataframe[dataframe['timestamp'].between(st_time,end_time)]
    
    neigh1 = timed_df[timed_df['from'] == node]
    
    neigh2 = timed_df[timed_df['from'].isin(neigh1['to'])]
    neigh2=neigh2[neigh2['from'] != node]
    
    neigh3 = timed_df[timed_df['from'].isin(neigh2['to'])]
    neigh3 = neigh3[~neigh3['from'].isin(neigh2['from'])]
    
    neigh4 = timed_df[timed_df['from'].isin(neigh3['to'])]
    neigh4 = neigh4[~neigh4['from'].isin(neigh3['from'])]
    
    return (neigh1,neigh2,neigh3,neigh4)
    
    


In [25]:
df.head()

,timestamp,from,to,value,gasLimit,gasPrice,gasUsed,0
0,1484475035,0,1890222,4.950000,3.9,2.0,2.1,mining
1,1484475035,1,1777,1.011962,9.0,2.0,2.1,mining
2,1484475035,1,20425,1.011957,9.0,2.0,2.1,mining
3,1484475035,1,1558,1.008363,9.0,2.0,2.1,mining
4,1484475035,1,15,1.016120,9.0,2.0,2.1,mining


In [26]:
df[df['from'] == 261]['timestamp'].iloc[0]

1484481572

In [27]:
st_time = 1484481572
duration = 700000
end_time = st_time+duration
addr = 261
(neigh1,neigh2,neigh3,neigh4)=send_extract_graph_h4_timed(addr,st_time,end_time,df)

In [23]:
df.iloc[-1]

timestamp    1499633540
from               1484
to              1642750
value          0.050131
gasLimit            9.0
gasPrice            2.0
gasUsed             2.1
0                mining
Name: 12846970, dtype: object

In [28]:
def aggregate_tx(data):
    selected_data = data[['from','to','value','gasLimit','gasPrice','gasUsed']].reset_index(drop=True)
    grouped_data = selected_data.groupby(['from','to'])
    
    agg_counts = grouped_data.size()
    agg_neigh = grouped_data.mean()
    
    agg_neigh['counts'] = agg_counts
    
    return agg_neigh.reset_index()

In [29]:
st_time = df.iloc[0]['timestamp']
duration = 700000
addr = 261
end_time = 0
while  end_time < df.iloc[-1]['timestamp']:
    end_time = st_time+duration
    (neigh1,neigh2,neigh3,neigh4)=send_extract_graph_h4_timed(addr,st_time,end_time,df)
    st_time = end_time
    
    print(neigh1.shape)
    break

(7324, 8)


In [28]:
neigh4.head()

,timestamp,from,to,value,gasLimit,gasPrice,gasUsed,0
124,1484475646,43,409407,0.082248,2.1001,2.0,2.1,exchange
249,1484476330,69,58,1.999580,2.1000,2.0,2.1,NaN
401,1484477134,69,58,9.989580,2.1000,2.0,2.1,NaN
450,1484477447,116,32824,55.719538,2.1000,2.0,2.1,NaN
514,1484477735,43,18773,0.354008,2.1001,2.0,2.1,exchange


In [29]:
agg_neigh = aggregate_tx(neigh4)
agg_neigh

,from,to,value,gasLimit,gasPrice,gasUsed,counts
0,43,179,0.165080,2.1001,2.0,2.1,1
1,43,191,0.006552,2.1001,2.0,2.1,1
2,43,255,0.014131,2.1001,2.0,2.1,23
3,43,336,24.827719,2.1001,2.0,2.1,1
4,43,1019,0.086800,2.1001,2.0,2.1,2
...,...,...,...,...,...,...,...
1326,19886,43,7.459580,2.1001,2.0,2.1,1
1327,19887,43,1.553140,2.1001,2.0,2.1,1
1328,19889,1890233,2.244580,2.1000,2.0,2.1,1
1329,19901,21467,9.272739,2.1000,4.1,2.1,1


In [30]:
neigh2_df = neigh4[['from','to','value']] #.reset_index(drop=True)
neigh2_df

,from,to,value
124,43,409407,0.082248
249,69,58,1.999580
401,69,58,9.989580
450,116,32824,55.719538
514,43,18773,0.354008
...,...,...,...
228828,2416,257,0.230078
228830,2958,290,0.013666
228857,19889,1890233,2.244580
229139,19901,21467,9.272739


In [31]:
grpby = neigh2_df.groupby(['from','to'])


In [32]:
grpby.aggregate(np.sum)

value
from  to                
43    179       0.165080
      191       0.006552
      255       0.325023
      336      24.827719
      1019      0.173601
...                  ...
19886 43        7.459580
19887 43        1.553140
19889 1890233   2.244580
19901 21467     9.272739
19932 147       2.262090

[1331 rows x 1 columns]

In [39]:
grpby.agg(['mean', 'count'])

value      
                    mean count
from  to                      
43    179       0.165080     1
      191       0.006552     1
      255       0.014131    23
      336      24.827719     1
      1019      0.086800     2
...                  ...   ...
19886 43        7.459580     1
19887 43        1.553140     1
19889 1890233   2.244580     1
19901 21467     9.272739     1
19932 147       2.262090     1

[1331 rows x 2 columns]

In [109]:
i = 0
for name, group in grpby:

    print(name)
    print(group)
    if i == 4:
        break
    i=i+1

(43, 179)
       from   to    value
76563    43  179  0.16508
(43, 191)
     from   to     value
875    43  191  0.006552
(43, 255)
        from   to     value
1215      43  255  0.010562
1225      43  255  0.011561
5629      43  255  0.019500
34458     43  255  0.011560
34547     43  255  0.011561
64574     43  255  0.014556
64681     43  255  0.014556
69132     43  255  0.019500
88314     43  255  0.009564
88319     43  255  0.009564
116254    43  255  0.015580
116387    43  255  0.015580
119377    43  255  0.021580
144414    43  255  0.012580
144470    43  255  0.012580
169016    43  255  0.012580
169039    43  255  0.012580
173419    43  255  0.021580
197402    43  255  0.010580
197705    43  255  0.011580
221422    43  255  0.013580
221522    43  255  0.013580
227626    43  255  0.018580
(43, 336)
      from   to      value
1623    43  336  24.827719
(43, 1019)
       from    to     value
5630     43  1019  0.031452
45144    43  1019  0.142149


In [43]:
df.iloc[-1]

timestamp    1499633540
from               1484
to              1642750
value          0.050131
gasLimit            9.0
gasPrice            2.0
gasUsed             2.1
0                mining
Name: 12846970, dtype: object

In [30]:
print(datetime.fromtimestamp(df.iloc[0]['timestamp']))
print(datetime.fromtimestamp(df.iloc[-1]['timestamp']))

2017-01-15 15:10:35
2017-07-10 01:52:20


In [31]:
def extract_graphs(addr,duration):
    st_time = df.iloc[0]['timestamp']
#     duration = 700000
#     addr = 261
    end_time = 0
    graphs = []
    while  end_time < df.iloc[-1]['timestamp']:
        end_time = st_time+duration
        (neigh1,neigh2,neigh3,neigh4)=send_extract_graph_h4_timed(addr,st_time,end_time,df)
        st_time = end_time
#         print(neigh1.shape)
        neigh1 = aggregate_tx(neigh1)
        neigh2 = aggregate_tx(neigh2)
        neigh3 = aggregate_tx(neigh3)
        neigh4 = aggregate_tx(neigh4)
        
        graphs.append((neigh1,neigh2,neigh3,neigh4))
    return graphs

In [32]:
miner_graph = []
duration = 700000
for addr in kept_mining_addrs:
    miner_graph.append(extract_graphs(addr,duration))

In [33]:
len(miner_graph[1])

22

In [29]:
(neigh1,neigh2,neigh3,neigh4) = miner_graph[0][0]

In [35]:
neigh4

,from,to,value,gasLimit,gasPrice,gasUsed,counts
0,43,179,0.165080,2.1001,2.0,2.1,1
1,43,191,0.006552,2.1001,2.0,2.1,1
2,43,255,0.014131,2.1001,2.0,2.1,23
3,43,336,24.827719,2.1001,2.0,2.1,1
4,43,1019,0.086800,2.1001,2.0,2.1,2
...,...,...,...,...,...,...,...
1805,19886,43,7.459580,2.1001,2.0,2.1,1
1806,19887,43,1.553140,2.1001,2.0,2.1,1
1807,19889,1890233,2.244580,2.1000,2.0,2.1,1
1808,19901,21467,9.272739,2.1000,4.1,2.1,1


In [34]:
ex_graph = []
duration = 700000
for addr in kept_ex_addrs:
    ex_graph.append(extract_graphs(addr,duration))

In [35]:
len(ex_graph[0])

22

In [43]:
(neigh1,neigh2,neigh3,neigh4) = ex_graph[0][0]

In [49]:

neigh1_nodes = pd.concat([neigh1['from'],neigh1['to']])
neigh2_nodes = pd.concat([neigh2['from'],neigh2['to']])
neigh3_nodes = pd.concat([neigh3['from'],neigh3['to']])
neigh4_nodes = pd.concat([neigh4['from'],neigh4['to']])

nodes = pd.concat([neigh1_nodes
                  ,neigh2_nodes
                  ,neigh3_nodes
                  ,neigh4_nodes]).unique()

In [52]:
map_id = {j:i for i,j in enumerate(nodes)}

In [53]:
neigh1['from'] = neigh1['from'].map(map_id)
neigh1['to'] = neigh1['to'].map(map_id)

In [55]:
neigh2['from'] = neigh2['from'].map(map_id)
neigh2['to'] = neigh2['to'].map(map_id)

In [56]:
neigh3['from'] = neigh3['from'].map(map_id)
neigh3['to'] = neigh3['to'].map(map_id)

In [57]:
neigh4['from'] = neigh4['from'].map(map_id)
neigh4['to'] = neigh4['to'].map(map_id)

In [59]:
neigh4.head()

,from,to,value,gasLimit,gasPrice,gasUsed,counts
0,9986,5090,0.01,9.0,2.0,2.1,1
1,9986,10437,0.01,9.0,2.0,2.1,1
2,9986,10438,0.01,9.0,2.0,2.1,1
3,9986,10439,0.01,9.0,2.0,2.1,1
4,9986,10440,0.01,9.0,2.0,2.1,1


In [36]:
import torch
from torch_geometric.data import Data

In [39]:
graph[['value','gasLimit','gasPrice','gasUsed' ,'counts']].head()

NameError: name 'graph' is not defined

In [83]:
graph = pd.concat([neigh1,neigh2,neigh3,neigh4])

# print(graph[['from','to']].to_numpy())
# print(graph)
graph[['from','to']].to_numpy().T.shape

(2, 17027)

In [75]:
edge_index = torch.tensor(graph[['from','to']].to_numpy().T, dtype=torch.long)
x = torch.ones(nodes.shape)

In [82]:
edge_attributes = graph[['value','gasLimit','gasPrice','gasUsed' ,'counts']].to_numpy()
edge_attributes.shape

(17027, 5)

In [87]:

data = Data(x=x, edge_index=edge_index,edge_attr = edge_attributes,y=[0])

In [37]:
def get_graph_data(graph,label):
    (neigh1,neigh2,neigh3,neigh4) = graph
    neigh1_nodes = pd.concat([neigh1['from'],neigh1['to']])
    neigh2_nodes = pd.concat([neigh2['from'],neigh2['to']])
    neigh3_nodes = pd.concat([neigh3['from'],neigh3['to']])
    neigh4_nodes = pd.concat([neigh4['from'],neigh4['to']])

    nodes = pd.concat([neigh1_nodes
                  ,neigh2_nodes
                  ,neigh3_nodes
                  ,neigh4_nodes]).unique()
    map_id = {j:i for i,j in enumerate(nodes)}
    
    neigh1['from'] = neigh1['from'].map(map_id)
    neigh1['to'] = neigh1['to'].map(map_id)
    
    neigh2['from'] = neigh2['from'].map(map_id)
    neigh2['to'] = neigh2['to'].map(map_id)
    
    neigh3['from'] = neigh3['from'].map(map_id)
    neigh3['to'] = neigh3['to'].map(map_id)
    
    neigh4['from'] = neigh4['from'].map(map_id)
    neigh4['to'] = neigh4['to'].map(map_id)
    
    graph = pd.concat([neigh1,neigh2,neigh3,neigh4])
    
    edge_index = torch.tensor(graph[['from','to']].to_numpy().T, dtype=torch.long)
    
    x = torch.ones(nodes.shape).reshape(-1,1)
    edge_attributes = graph[['value','gasLimit','gasPrice','gasUsed' ,'counts']].to_numpy()
    
#     print(label)
    data = Data(x=x, edge_index=edge_index,edge_attr = edge_attributes,y=torch.tensor([label]))
    
    return data
data = get_graph_data(ex_graph[0][0],0)
data.x.reshape(-1,1).shape

torch.Size([12995, 1])

In [38]:
data_list = []
def add_graphs_to_list(graphs,label):
    for addr_graphs in graphs:
        for graph in addr_graphs:
            data_list.append(get_graph_data(graph,label))
            

add_graphs_to_list(ex_graph,0)
add_graphs_to_list(miner_graph,1)

In [36]:
len(data_list)

924

In [39]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.data import InMemoryDataset
loader = DataLoader(data_list, batch_size=32)


In [40]:
class MyDataset(InMemoryDataset):
    def __init__(self, root, data_list, transform=None):
        self.data_list = data_list
        super().__init__(root, transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def processed_file_names(self):
        return 'data.pt'

    def process(self):
        torch.save(self.collate(self.data_list), self.processed_paths[0])

In [41]:
mydataset = MyDataset('./data',data_list)

Processing...
Done!


In [51]:
!mkdir data

In [65]:
# loader = DataLoader(mydataset, batch_size=32)
train_loader = DataLoader(mydataset, batch_size=32, shuffle=True)

In [66]:
print(f'Dataset: {mydataset}:')
print('====================')
print(f'Number of graphs: {len(mydataset)}')
print(f'Number of features: {mydataset.num_features}')
print(f'Number of classes: {mydataset.num_classes}')

Dataset: MyDataset(924):
Number of graphs: 924
Number of features: 1
Number of classes: 2


In [67]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(mydataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, mydataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64).to(device)
print(model)

GCN(
  (conv1): GCNConv(1, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [68]:
model = GCN(hidden_channels=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [106]:
def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        data = data.to(device)
#         print(data.shape)
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
#         print(out.shape)
#         break
        if out.shape[0] != data.y.shape[0]:
#             print((out.shape,data.y.shape))
            continue
#         print((out.shape[0],data.y.shape[0]))
#         break
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
            
def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        if pred.shape[0]!=data.y.shape[0]:
            continue
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.


In [107]:

for epoch in range(1, 200):
    train()
#     break
    train_acc = test(train_loader)
#     test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')

RuntimeError: CUDA out of memory. Tried to allocate 498.00 MiB (GPU 0; 11.91 GiB total capacity; 9.39 GiB already allocated; 365.69 MiB free; 10.40 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [108]:
!nvidia-smi

Fri Mar  4 17:05:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN Xp     Off  | 00000000:01:00.0  On |                  N/A |
| 23%   38C    P8    11W / 250W |  11835MiB / 12194MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------